## Setup

Requires a HubSpot Private app and an API access token with the following scopes:

- `crm.schemas.custom.read`
- `crm.objects.custom.read`
- `crm.objects.custom.write`
- `crm.objects.companies.read`
- `crm.schemas.contacts.read`
- `crm.objects.contacts.read`
- `crm.schemas.companies.read`
- `sales-email-read`

In [ ]:
import os

from hubspot import HubSpot
import pandas as pd


ACCESS_TOKEN = os.environ["HUBSPOT_ACCESS_TOKEN"]
ASSOCIATION_TYPES = ["calls", "emails", "meetings", "notes", "tasks"]

hubspot = HubSpot(access_token=ACCESS_TOKEN)

In [ ]:
def write_json_records(df: pd.DataFrame, file_path: str):
    """Helper writes the DataFrame into a JSON file."""
    df.to_json(f"data/{file_path}", orient="records", indent=2)

In [ ]:
# request company properties
company_props = hubspot.crm.properties.core_api.get_all(object_type="companies", archived=False)

In [ ]:
# read company properties into DataFrame
company_props_df = pd.json_normalize(company_props.to_dict(), "results")
write_json_records(company_props_df, "company_props.json")

In [ ]:
# request companies data
companies = hubspot.crm.companies.get_all(properties=["company_type", "domain", "name"], associations=ASSOCIATION_TYPES)
companies = [c.to_dict() for c in companies]

In [ ]:
# read companies data into DataFrame
companies_df = pd.json_normalize(companies)
companies_df["properties.domain"] = companies_df["properties.domain"].astype("category")
write_json_records(companies_df, "companies.json")

In [ ]:
# look at the unique company types defined
companies_df["properties.company_type"].unique().tofile("data/company_types.txt", sep=os.linesep)

In [ ]:
# filter vendor companies into new DataFrame
vendor_companies_df = companies_df[companies_df["properties.company_type"] == "Vendor"]
write_json_records(vendor_companies_df, "companies_vendors.json")

In [ ]:
# request vendor properties
vendor_props = hubspot.crm.properties.core_api.get_all(object_type="vendors", archived=False)

In [ ]:
# read vendor properties into DataFrame
vendor_props_df = pd.json_normalize(vendor_props.to_dict(), "results")
write_json_records(vendor_props_df, "vendor_props.json")

In [ ]:
# request vendor data
vendors = hubspot.crm.objects.get_all("vendors", properties=["domain", "vendor_name"])
vendors = [v.to_dict() for v in vendors]

In [ ]:
# read vendor data into DataFrame
vendors_df = pd.json_normalize(vendors)
vendors_df["properties.domain"] = vendors_df["properties.domain"].astype("category")
write_json_records(vendors_df, "vendors.json")

# filter any custom object vendors missing their domain property
# these were used for testing the vendor object import
vendors_df = vendors_df[~vendors_df["properties.domain"].isna()]
write_json_records(vendors_df, "vendors_with_domains.json")

In [ ]:
# confirming associations exist for the vendor custom object
vendor_associations_df = pd.DataFrame(columns=["name", "id"])
for association_type in ASSOCIATION_TYPES:
    vendor_associations = hubspot.crm.associations.schema.types_api.get_all("vendors", association_type)
    df = pd.json_normalize(vendor_associations.to_dict(), "results")
    vendor_associations_df = vendor_associations_df.merge(df, how="outer")
write_json_records(vendor_associations_df, "vendor_associations.json")

In [ ]:
# confirming reverse associations exist for the vendor custom object
reverse_vendor_associations_df = pd.DataFrame(columns=["name", "id"])
for association_type in ASSOCIATION_TYPES:
    reverse_vendor_associations = hubspot.crm.associations.schema.types_api.get_all(association_type, "vendors")
    df = pd.json_normalize(reverse_vendor_associations.to_dict(), "results")
    reverse_vendor_associations_df = reverse_vendor_associations_df.merge(df, how="outer")
write_json_records(reverse_vendor_associations_df, f"vendor_associations_reverse.json")

In [ ]:
# combine the company vendors and custom object vendors into a single DataFrame with all columns
# using an LEFT JOIN on domain
# keeps records that have at least company vendor details
joined_vendors_df_domain = vendor_companies_df.merge(vendors_df, on="properties.domain", how="left", suffixes=("_company", "_custom"))

# filter mismatched companies for those with any activities
missing_custom_with_activity_criteria_domain = (
    joined_vendors_df_domain["id_custom"].isna() &
    any([joined_vendors_df_domain[f"associations.{a}.results"].count() > 0 for a in ASSOCIATION_TYPES])
)
missing_custom_with_activity_domain = joined_vendors_df_domain[missing_custom_with_activity_criteria_domain]
# sort by name
missing_custom_with_activity_domain = missing_custom_with_activity_domain.sort_values("properties.name")
# rename joined columns
renames_domain = {"id_company": "id", "properties.hs_object_id_company": "properties.hs_object_id"}
renamed_missing_domain = missing_custom_with_activity_domain.rename(columns=renames_domain)
# select just the columns for later joining
select_domain = list(renames_domain.values()) + ["properties.company_type", "properties.domain", "properties.name"]
missing_output_domain = renamed_missing_domain[select_domain]

# combine the company vendors and custom object vendors into a single DataFrame with all columns
# using an LEFT JOIN on name
# keeps records that have at least company vendor details
joined_vendors_df_name = vendor_companies_df.merge(
    vendors_df, left_on="properties.name", right_on="properties.vendor_name", how="left", suffixes=("_company", "_custom")
)

# filter mismatched companies for those with any activities
missing_custom_with_activity_criteria_name = (
    joined_vendors_df_name["id_custom"].isna() &
    any([joined_vendors_df_name[f"associations.{a}.results"].count() > 0 for a in ASSOCIATION_TYPES])
)
missing_custom_with_activity_name = joined_vendors_df_name[missing_custom_with_activity_criteria_name]
# sort by name
missing_custom_with_activity_name = missing_custom_with_activity_name.sort_values("properties.name")
# rename joined columns
renames_name = {"id_company": "id", "properties.hs_object_id_company": "properties.hs_object_id", "properties.domain_company": "properties.domain"}
renamed_missing_name = missing_custom_with_activity_name.rename(columns=renames_name)
# select just the columns for later joining
select_name = list(renames_name.values()) + ["properties.company_type", "properties.name"]
missing_name = renamed_missing_name[select_name]

# merge the two DataFrames together
# these are all the company vendors with activities that didn't have a matching custom vendor object
# either on domain or name
merged_missing = missing_output_domain.merge(missing_name, how="outer", on="properties.domain")
# sort the columns
sorted_cols = sorted(merged_missing.columns.to_list())
merged_missing = merged_missing.reindex(columns=sorted_cols)
# backfill missing values from the nearest column
# since the columns are sorted, when e.g. column_x is missing, it will be filled from column_y
merged_missing = merged_missing.replace("", pd.NA).bfill(axis=1)
# rename joined columns now that there is a value for each
renames = {"properties.name_x": "properties.name", "properties.hs_object_id_x": "properties.hs_object_id", "id_x": "id", "properties.company_type_x": "properties.company_type"}
merged_missing = merged_missing.rename(columns=renames)
# select a limited list of columns for output
select = ["properties.domain"] + list(renames.values())
selected_output = merged_missing[select]
# filter companies already accounted for with domain mismatch
selected_output = selected_output[~selected_output["properties.name"].isin(["Hayden AI", "Kontron", "RideCo", "Peak Transit"])]
write_json_records(selected_output, "vendors_join_left_merged_missing_custom_with_activity.json")
# adds a separator row for CSV output into markdown table format
separator_row = pd.DataFrame([map(lambda x: "-----", select)], columns=select)
csv_output = pd.concat([separator_row, selected_output], ignore_index=True)
csv_output.replace("|", "\|", inplace=True)
csv_output.to_csv("data/vendors_join_left_merged_missing_custom_with_activity.csv", index=False, sep="|")